In [1]:
import os
import subprocess as sp
import re
import requests

import googlesearch as gs
from bs4 import BeautifulSoup
from openai import OpenAI


openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ReAct (Reason + Act) Framework (Yao et al, 2022)
- Thought
- Action
- Pause
- Observation
- Answer

In [2]:
system_prompt = """
You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return Pause.
Observation will be the result of running those actions.

Your available actions are:
ping:
e.g. ping: python.org
Does a ping command and return the response time in seconds

bash:
e.g. bash: python --version
Returns the result of bash command execution

web_search:
e.g. web_search: capital of Portugal
Returns the content of the first result of a google search

Example session:
Question: How many islands make up Madeira?
Thought: I should do a web search for the Madeira
Action: web_search: Madeira
Pause

You will be called again with this:
Observation: Madeira is a Portuguese island chain made up of four islands: Madeira, Porto Santo, Desertas, and Selvagens, only two of which are inhabited (Madeira and Porto Santo.) 

You then output:
Answer: Four islands
"""

In [3]:
def query_model(messages, model="gpt-4o"):
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [4]:
def ping(website: str):
    if not website.startswith("https://"):
        website = "https://" + website
    response = requests.get(website)
    return response.elapsed.total_seconds()


def bash(command: str):
    out = sp.check_output(command, shell=True)
    return out.decode("utf-8").rstrip()


def web_search(query: str):
    top_result = next(gs.search(query, advanced=True))
    print("Using data from: ", top_result.url)
    content = requests.get(top_result.url)
    return re.sub(" {2,}", "", BeautifulSoup(content.text, "lxml").text.replace("\n", "").replace("\r", ""))

In [5]:
ping("heapcon.io")

0.661907

In [6]:
bash("python --version")

'Python 3.12.6'

In [7]:
web_search("Capital of Serbia")

Using data from:  https://en.wikipedia.org/wiki/Belgrade


'Belgrade - WikipediaJump to contentMain menuMain menumove to sidebarhide\t\tNavigation\tMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\t\tContribute\tHelpLearn to editCommunity portalRecent changesUpload fileSearchSearchAppearanceDonateCreate accountLog inPersonal toolsDonate Create account Log in\t\tPages for logged out editors learn moreContributionsTalkContentsmove to sidebarhide(Top)1HistoryToggle History subsection1.1Prehistory1.2Antiquity1.3Middle Ages1.4Ottoman rule and Austrian invasions1.5Principality and Kingdom of Serbia1.6World War I: Austro–German invasion1.7Kingdom of Yugoslavia1.8World War II: German invasion1.9Socialist Yugoslavia1.10Breakup of Yugoslavia1.11Development2GeographyToggle Geography subsection2.1Topography2.2Climate3AdministrationToggle Administration subsection3.1Municipalities4Demographics5Economy6CultureToggle Culture subsection6.1Museums6.2Architecture6.3Tourism6.4Nightlife6.5Sport and recreation6.6Fashion and design7Media8Educa

In [8]:
known_actions = {
    "ping": ping,
    "bash": bash,
    "web_search": web_search
}

In [9]:
def query(user_query: str, max_iter: int = 5):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    counter = 1
    while counter < max_iter:
        print(f"Loop: {counter}")
        response = query_model(messages)
        print(response)
        actions = [re.match(r"^Action: (\w+): (.*)", a) for a in response.split('\n') if re.match(r"^Action: (\w+): (.*)", a)]
        if actions:
            action, action_input = actions[0].groups()
            print("Running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
            messages.append({"role": "user", "content": next_prompt})
        else:
             break
        counter += 1

In [10]:
user_prompt = "What's the response time for google.com?"
query(user_prompt)

Loop: 1
Thought: To find the response time for google.com, I should perform a ping command for the domain.
Action: ping: google.com
Pause
Running ping google.com
Observation: 0.55325
Loop: 2
Answer: The response time for google.com is approximately 0.553 seconds.


In [11]:
user_prompt = "What python packages do I have installed?"
query(user_prompt)

Loop: 1
To find out which Python packages are installed, I will run the appropriate bash command to list all installed Python packages. 

Action: bash: pip list
Pause
Running bash pip list
Observation: Package                   Version
------------------------- --------------
annotated-types           0.7.0
anyio                     4.6.1
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
comm                      0.2.2
debugpy                   1.8.7
decorator                 5.1.1
defusedxml                0.7.1
distro                    1.9.0
executing                 2.1.0
fastjsonschema            2.20.0
fqdn    

In [12]:
user_prompt = "What's new in nodejs 23?"
query(user_prompt)

Loop: 1
Thought: To provide accurate and up-to-date information on what's new in Node.js 23, I should perform a web search for the latest details on the release. 
Action: web_search: what's new in Node.js 23
Pause.
Running web_search what's new in Node.js 23
Using data from:  https://nodejs.org/en/blog/release/v23.0.0
Observation: Node.js — Node v23.0.0 (Current)LearnAboutDownloadBlogDocsCertificationStart typing...Ctrl KNode v23.0.0 (Current)Rafael GonzagaNode v23.0.0 (Current)2024-10-16, Version 23.0.0 (Current), @RafaelGSSWe’re excited to announce the release of Node.js 23! Key highlights include:Enabling require(esm) by default for Node.js applicationsRemoving support for Windows 32-bit systemsStabilizing the node --run commandEnhancements to the test runner, including glob pattern support for coverage filesNode.js 23 will replace Node.js 22 as the ‘Current’ release line when Node.js 22 enters long-term support (LTS) later this month.According to the release schedule, Node.js 23 wi

In [13]:
user_prompt = "Who are the speakers at Heapcon 2024?"
query(user_prompt)

Loop: 1
Thought: I should perform a web search to find the list of speakers for Heapcon 2024. 
Action: web_search: Heapcon 2024 speakers list
Pause
Running web_search Heapcon 2024 speakers list
Using data from:  https://heapcon.io/2024/speakers
Observation:  Speakers | Heapcon 2024[ Tickets ][ Speakers ][ Agenda ][ Sponsors ][ Tickets ][ Speakers ][ Agenda ][ Sponsors ]>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[ Venue ][ Team ][ About ][ Committee ][ Blog ]>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Past_conferences[ Heapcon_2018 ][ Heapcon_2019 ][ Heapcon_2022 ][ Heapcon_2023 ]<headline>A mix of world-renowned speakers and regional experts, all under one roof.</headline><Speakers_lineup> 